# 🚗 Car Video to 3D Model using TRELLIS

This notebook converts your car video frames into a high-quality 3D model (.glb) that you can use on your website.

**Steps:**
1. Run all cells in order
2. Upload your frames ZIP file when prompted
3. Wait for processing (5-15 minutes)
4. Download the generated .glb file

**Requirements:**
- GPU Runtime (Go to Runtime > Change runtime type > GPU)

In [ ]:
# Step 1: Install TRELLIS and dependencies
print("📦 Installing TRELLIS...")
!git clone --recurse-submodules https://github.com/microsoft/TRELLIS.git
%cd TRELLIS

# Install dependencies (modified for Colab)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers diffusers accelerate pillow imageio imageio-ffmpeg
!pip install -q gradio omegaconf einops

print("✅ Installation complete!")

In [ ]:
# Step 2: Upload your frames
from google.colab import files
import zipfile
import os

print("📤 Please upload your car_frames.zip file")
uploaded = files.upload()

# Extract frames
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📂 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('frames')
        print("✅ Frames extracted!")

# List frames
frames = sorted([f for f in os.listdir('frames') if f.endswith(('.png', '.jpg'))])
print(f"Found {len(frames)} frames")

In [ ]:
# Step 3: Load TRELLIS model
import os
os.environ['SPCONV_ALGO'] = 'native'

from PIL import Image
from trellis.pipelines import TrellisImageTo3DPipeline
from trellis.utils import render_utils, postprocessing_utils

print("🤖 Loading TRELLIS model...")
pipeline = TrellisImageTo3DPipeline.from_pretrained("microsoft/TRELLIS-image-large")
pipeline.cuda()
print("✅ Model loaded!")

In [ ]:
# Step 4: Select best frame (middle frame or you can change this)
import numpy as np

# Use the middle frame as the primary view
middle_idx = len(frames) // 2
primary_frame = Image.open(f'frames/{frames[middle_idx]}')

print(f"Using frame: {frames[middle_idx]}")
display(primary_frame.resize((256, 256)))  # Preview

In [ ]:
# Step 5: Generate 3D model
print("🎨 Generating 3D model... (this may take 5-15 minutes)")

outputs = pipeline.run(
    primary_frame,
    seed=42,
    sparse_structure_sampler_params={
        "steps": 12,
        "cfg_strength": 7.5,
    },
    slat_sampler_params={
        "steps": 12,
        "cfg_strength": 3,
    },
)

print("✅ 3D model generated!")

In [ ]:
# Step 6: Export as GLB file
import imageio

print("💾 Exporting GLB file...")

# Create preview video
video = render_utils.render_video(outputs['mesh'][0])['normal']
imageio.mimsave("preview.mp4", video, fps=30)

# Export GLB
glb = postprocessing_utils.to_glb(
    outputs['gaussian'][0],
    outputs['mesh'][0],
    simplify=0.95,
    texture_size=2048,  # High quality texture
)

output_filename = "car_model_3d.glb"
glb.export(output_filename)

print(f"✅ GLB file created: {output_filename}")
print("📥 Downloading files...")

# Download both preview and GLB
files.download("preview.mp4")
files.download(output_filename)

print("")
print("🎉 DONE! Next steps:")
print("1. Place car_model_3d.glb in your project's public/models/ folder")
print("2. Update data.js with: model3d: '/models/car_model_3d.glb'")
print("3. Refresh your website to see the interactive 3D model!")